In [70]:
!pwd
!ls

/d/GH/GitWorkSpace/bank_model_competiton/data/v10
model_stacking_v10.1.ipynb
process_v10.ipynb
test.dat.v10
test.dat.v9
train.dat.v10
train.dat.v9


In [71]:
import pandas as pd 
import matplotlib.pyplot as plt
import statistics
import datetime
import seaborn as sns
import os
import numpy as np
import time 
from sklearn.preprocessing import OrdinalEncoder


plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

suffix = os.path.split(os.getcwd())[-1]

root_dir='../../'

train_path=root_dir+'train.csv'
train_tx_path=root_dir+'train_bank_statement.csv'
test_path=root_dir+'testaa.csv'
test_tx_path=root_dir+'testaa_bank_statement.csv'

print(suffix)
print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))

v10
process time :  2025-08-28 13:47:02


In [72]:

MODE="test"
NROWS = 10000000000000000

if MODE == "train":
    base_input_path = train_path
    tx_input_path = train_tx_path
    output_path = "train.dat.%s" % suffix
else:
    base_input_path = test_path
    tx_input_path = test_tx_path
    output_path = "test.dat.%s" % suffix

print(mode, base_input_path, tx_input_path, output_path)

test ../../testaa.csv ../../testaa_bank_statement.csv test.dat.v10


#  基础特征

In [73]:
!pwd
df = pd.read_csv(base_input_path, index_col=['id'], nrows=NROWS)
print(df.head())
print(df.shape)
pd.set_option('display.max_columns', None)

df['balance_account_avg'] = df['balance'] / df['balance_accounts']
df['loan_term_avg'] = df['loan'] / df['term']
df['balance_accounts_ratio'] = df['balance_accounts'] / df['total_accounts']

df['loan_log'] = df['loan'].apply(lambda x : np.log(x))

#log 处理
df['balance_account_avg_log'] = df['balance_account_avg'].apply(lambda x : np.log(x))
df['loan_term_avg_log'] = df['loan_term_avg'].apply(lambda x : np.log(x))
df['balance_accounts_ratio_log'] = df['balance_accounts_ratio'].apply(lambda x : np.log(x))
df['interest_rate_log'] = df['interest_rate'].apply(lambda x : np.log(x))

df['balance_log'] = df['balance'].apply(lambda x : np.log(x))
df['balance_limit_log'] = df['balance_limit'].apply(lambda x : np.log(x))
df['balance_accounts_log'] = df['balance_accounts'].apply(lambda x : np.log(x))

# interest_rate_labels = ['%d' % i for i in range(20)]
# print(interest_rate_labels)
# df['interest_rate_cut'] = pd.cut(df['interest_rate'], bins=20, labels = interest_rate_labels)

#level 特征
df['level_hash'] = df['level'].apply(lambda x : hash(x) % 1000 )

encoder = OrdinalEncoder(categories=[['A0','A1','A2','A3','A4','A5','B0','B1','B2','B3','B4','B5','C0','C1','C2','C3','C4','C5','D0','D1','D2','D3','D4','D5','E0','E1','E2','E3','E4','E5']]) 
df['level_ord'] = encoder.fit_transform(df[['level']].values)

# 时间处理 
df['record_issue_diff'] =  df['record_time'] - df['issue_time']
df['record_issue_days'] =  df['record_issue_diff']/(60*60*24)

df['record_history_diff'] =  df['record_time'] - df['history_time']
df['record_history_days'] =  df['record_issue_diff']/(60*60*24)

df['issue_history_diff'] =  df['issue_time'] - df['history_time']
df['issue_history_days'] =  df['issue_history_diff']/(60*60*24)

df

/d/GH/GitWorkSpace/bank_model_competiton/data/v10
       title  career  zip_code  residence   loan  term  interest_rate  \
id                                                                      
53480     10     5.0    512068          0   5950    36          14.10   
53481      8    10.0    401852          2  15350    36           5.25   
53482      8    10.0    321693          0  11050    36          13.65   
53483      8     7.0    231397          0  15350    36          14.20   
53484      7     7.0    171193          0   7800    36          10.00   

       issue_time  syndicated  installment  record_time  history_time  \
id                                                                      
53480  1054600001           0            0   1054600080     795400642   
53481  1172880047           0            0   1172882338     756003297   
53482  1238631003           0            0   1277511093     658715225   
53483  1136161107           0            0   1198371199     774316843   


,title,career,zip_code,residence,loan,term,interest_rate,issue_time,syndicated,installment,record_time,history_time,total_accounts,balance_accounts,balance_limit,balance,level,balance_account_avg,loan_term_avg,balance_accounts_ratio,loan_log,balance_account_avg_log,loan_term_avg_log,balance_accounts_ratio_log,interest_rate_log,balance_log,balance_limit_log,balance_accounts_log,level_hash,level_ord,record_issue_diff,record_issue_days,record_history_diff,record_history_days,issue_history_diff,issue_history_days
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
53480,10,5.0,512068,0,5950,36,14.10,1054600001,0,0,1054600080,795400642,17,9,10100.0,8233.00,B0,914.777778,165.277778,0.529412,8.691146,6.818681,5.107628,-0.635989,2.646175,9.015906,9.220291,2.197225,310,6.0,79,0.000914,259199438,0.000914,259199359,2999.992581
53481,8,10.0,401852,2,15350,36,5.25,1172880047,0,0,1172882338,756003297,17,9,58600.0,23460.00,A0,2606.666667,426.388889,0.529412,9.638871,7.865828,6.055352,-0.635989,1.658228,10.063052,10.978490,2.197225,630,0.0,2291,0.026516,416879041,0.026516,416876750,4824.962384
53482,8,10.0,321693,0,11050,36,13.65,1238631003,0,0,1277511093,658715225,17,9,15500.0,7853.00,B1,872.555556,306.944444,0.529412,9.310186,6.771426,5.726667,-0.635989,2.613740,8.968651,9.648595,2.197225,594,7.0,38880090,450.001042,618795868,450.001042,579915778,6711.988171
53483,8,7.0,231397,0,15350,36,14.20,1136161107,0,0,1198371199,774316843,17,9,10100.0,5815.00,B2,646.111111,426.388889,0.529412,9.638871,6.470971,6.055352,-0.635989,2.653242,8.668196,9.220291,2.197225,288,8.0,62210092,720.024213,424054356,720.024213,361844264,4188.012315
53484,7,7.0,171193,0,7800,36,10.00,1214960354,0,1,1214957804,832207268,17,9,13100.0,8225.00,A3,913.888889,216.666667,0.529412,8.961879,6.817709,5.378360,-0.635989,2.302585,9.014934,9.480368,2.197225,17,3.0,-2550,-0.029514,382750536,-0.029514,382753086,4430.012569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73529,0,8.0,601107,1,10000,12,18.85,1130976000,0,0,1125964800,1018224000,6,3,3818.0,2224.69,A4,741.563333,833.333333,0.500000,9.210340,6.608761,6.725434,-0.693147,2.936513,7.707373,8.247482,1.098612,160,4.0,-5011200,-58.000000,107740800,-58.000000,112752000,1305.000000
73530,0,10.0,601102,1,10000,12,29.30,1156204800,0,0,1157068800,1054425600,6,6,5502.0,4126.71,B4,687.785000,833.333333,1.000000,9.210340,6.533476,6.725434,0.000000,3.377588,8.325236,8.612867,1.791759,449,10.0,864000,10.000000,102643200,10.000000,101779200,1178.000000
73531,0,4.0,601408,1,11000,12,24.75,1144108800,0,0,1111622400,1037404800,8,3,4844.0,2710.96,A3,903.653333,916.666667,0.375000,9.305651,6.806446,6.820744,-0.980829,3.208825,7.905058,8.485496,1.098612,17,3.0,-32486400,-376.000000,74217600,-376.000000,106704000,1235.000000


#  交易特征处理

In [74]:
df_tx = pd.read_csv(tx_input_path, nrows = NROWS, index_col=['id'])
df_tx['time_format'] = df_tx['time'].apply(lambda x: datetime.datetime.fromtimestamp(x))

print(df['record_time'])
print(df_tx)

# 假设 ：issue time 是记录违约的时间，那只能用在此之前的流水
# df_tx_merge = pd.merge(df_tx, df[['record_time','issue_time']], on = 'id', how='left')
# df_tx = df_tx_merge[ df_tx_merge['time'] <  df_tx_merge['issue_time'] ]

id
53480    1054600080
53481    1172882338
53482    1277511093
53483    1198371199
53484    1214957804
            ...    
73529    1125964800
73530    1157068800
73531    1111622400
73532    1116892800
73533    1146614400
Name: record_time, Length: 20054, dtype: int64
             time  direction       amount         time_format
id                                                           
56693  1144195200          1   542.210000 2006-04-05 08:00:00
56693  1145145600          1   283.490000 2006-04-16 08:00:00
56693  1145145600          0  5604.080000 2006-04-16 08:00:00
56693  1146787200          1   539.380000 2006-05-05 08:00:00
56693  1147737600          0  5951.750000 2006-05-16 08:00:00
...           ...        ...          ...                 ...
71870  1160956800          1   493.403945 2006-10-16 08:00:00
71870  1161043200          0     9.462382 2006-10-17 08:00:00
71870  1161388800          1   222.936072 2006-10-21 08:00:00
71870  1161475200          0   222.936072 2006-1

In [75]:
#days_diff, tm_count, total_amount, amount_1, amount_0, total_amount_day_avg, amount_1_day_avg, amount_0_day_avg
print(df_tx.shape)

df_tx_stat = pd.DataFrame()
df_tx_stat['tx_tmstp_max'] = df_tx.groupby('id')['time'].agg('max')
df_tx_stat['tx_tmstp_min'] = df_tx.groupby('id')['time'].agg('min')

df_tx_stat['tx_time_max'] = df_tx.groupby('id')['time_format'].agg('max')
df_tx_stat['tx_time_min'] = df_tx.groupby('id')['time_format'].agg('min')

df_tx_stat['tx_max_min_days'] = (df_tx_stat['tx_time_max'] - df_tx_stat['tx_time_min'])
df_tx_stat['tx_max_min_days'] = df_tx_stat['tx_max_min_days'].map(lambda x : x.days)

df_tx_stat['tx_count'] = df_tx.groupby('id')['amount'].agg('count')
df_tx_stat['total_amount'] = df_tx.groupby('id')['amount'].agg('sum')

df_tx_stat['1_amount'] = df_tx.groupby('id').apply( lambda x : x [ x['direction'] == 1]['amount'].sum())
df_tx_stat['0_amount'] = df_tx.groupby('id').apply( lambda x : x [ x['direction'] == 0]['amount'].sum())

#交易活跃度： 效果次数/天数
df_tx_stat['tx_count_avg']  = df_tx_stat['tx_count'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['total_amount_avg']  = df_tx_stat['total_amount'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['1_amount_avg']  = df_tx_stat['1_amount'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['0_amount_avg']  = df_tx_stat['0_amount'] / df_tx_stat['tx_max_min_days'] 

df_tx_stat['total_amount_avg2']  = df_tx_stat['total_amount'] / df_tx_stat['tx_count'] 
df_tx_stat['1_amount_avg2']  = df_tx_stat['1_amount'] / df_tx_stat['tx_count'] 
df_tx_stat['0_amount_avg2']  = df_tx_stat['0_amount'] / df_tx_stat['tx_count'] 

#log处理
df_tx_stat['tx_count_log'] = df_tx_stat['tx_count'].apply(lambda x : np.log(x))
df_tx_stat['total_amount_log'] = df_tx_stat['total_amount'].apply(lambda x : np.log(x))

df_tx_stat['1_amount_log'] = df_tx_stat['1_amount'].apply(lambda x : np.log(x))
df_tx_stat['0_amount_log'] = df_tx_stat['0_amount'].apply(lambda x : np.log(x))

df_tx_stat['tx_count_avg_log'] = df_tx_stat['tx_count_avg'].apply(lambda x : np.log(x))
df_tx_stat['total_amount_avg_log'] = df_tx_stat['total_amount_avg'].apply(lambda x : np.log(x))
df_tx_stat['1_amount_avg_log'] = df_tx_stat['1_amount_avg'].apply(lambda x : np.log(x))
df_tx_stat['0_amount_avg_log'] = df_tx_stat['0_amount_avg'].apply(lambda x : np.log(x))

df_tx_stat['total_amount_avg2_log'] = df_tx_stat['total_amount_avg2'].apply(lambda x : np.log(x))
df_tx_stat['1_amount_avg2_log'] = df_tx_stat['1_amount_avg2'].apply(lambda x : np.log(x))
df_tx_stat['0_amount_avg2_log'] = df_tx_stat['0_amount_avg2'].apply(lambda x : np.log(x))
df_tx_stat['tx_count_log_log'] = df_tx_stat['tx_count'].apply(lambda x : np.log(x))


df_tx_stat.drop([ 'tx_time_max', 'tx_time_min'], axis = 1, inplace=True)
df_tx_stat

(646666, 4)


,tx_tmstp_max,tx_tmstp_min,tx_max_min_days,tx_count,total_amount,1_amount,0_amount,tx_count_avg,total_amount_avg,1_amount_avg,0_amount_avg,total_amount_avg2,1_amount_avg2,0_amount_avg2,tx_count_log,total_amount_log,1_amount_log,0_amount_log,tx_count_avg_log,total_amount_avg_log,1_amount_avg_log,0_amount_avg_log,total_amount_avg2_log,1_amount_avg2_log,0_amount_avg2_log,tx_count_log_log
id,,,,,,,,,,,,,,,,,,,,,,,,,,
53480,1053302400,1039046400,165,46,61815.520000,11245.520000,50570.000000,0.278788,374.639515,68.154667,306.484848,1343.815652,244.467826,1099.347826,3.828641,11.031910,9.327725,10.831114,-1.277304,5.925964,4.221780,5.725168,7.203268,5.499084,7.002472,3.828641
53481,1170806400,1158364800,144,56,38555.840000,20161.940000,18393.900000,0.388889,267.748889,140.013472,127.735417,688.497143,360.034643,328.462500,4.025352,10.559863,9.911552,9.819774,-0.944462,5.590050,4.941739,4.849961,6.534511,5.886200,5.794423,4.025352
53482,1275350400,1260489600,172,74,81589.800000,44977.460000,36612.340000,0.430233,474.359302,261.496860,212.862442,1102.564865,607.803514,494.761351,4.304065,11.309460,10.713917,10.508141,-0.843429,6.161965,5.566422,5.360646,7.005394,6.409852,6.204076,4.304065
53483,1195776000,1182038400,159,73,48205.320000,24048.520000,24156.800000,0.459119,303.178113,151.248553,151.929560,660.346849,329.431781,330.915068,4.290459,10.783225,10.087829,10.092321,-0.778445,5.714320,5.018925,5.023417,6.492765,5.797369,5.801862,4.290459
53484,1214438400,1199404800,174,91,78113.610000,29797.860000,48315.750000,0.522989,448.928793,171.252069,277.676724,858.391319,327.449011,530.942308,4.510860,11.265920,10.302192,10.785513,-0.648196,6.106864,5.143137,5.626458,6.755060,5.791332,6.274653,4.510860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73515,1161475200,1153353600,94,25,9477.597620,5109.004148,4368.593472,0.265957,100.825507,54.351108,46.474399,379.103905,204.360166,174.743739,3.218876,9.156686,8.538760,8.382196,-1.324419,4.613391,3.995465,3.838902,5.937810,5.319884,5.163321,3.218876
73517,1161475200,1145923200,180,272,212302.562407,126112.342194,86190.220212,1.511111,1179.458680,700.624123,478.834557,780.524126,463.648317,316.875810,5.605802,12.265768,11.744928,11.364312,0.412845,7.072811,6.551972,6.171355,6.659966,6.139126,5.758510,5.605802
73522,1161475200,1145923200,180,116,45701.276358,24957.995111,20743.281247,0.644444,253.895980,138.655528,115.240451,393.976520,215.155130,178.821390,4.753590,10.729882,10.124949,9.939978,-0.439367,5.536925,4.931993,4.747021,5.976291,5.371359,5.186387,4.753590


In [76]:
df_tx_stat.shape

(8180, 26)

# 合并交易特征

In [77]:
df_concat = pd.concat([df, df_tx_stat], axis=1)
print(df_concat.shape)
df_concat.reset_index(inplace=True)

(20054, 62)


In [78]:
df_concat = df_concat.rename(columns={'index':'id'})
print(df_concat)
df_concat.to_csv(output_path, index=False)
print('output_path : ', output_path)
print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))
!ls

          id  title  career  zip_code  residence   loan  term  interest_rate  \
0      53480     10     5.0    512068          0   5950    36          14.10   
1      53481      8    10.0    401852          2  15350    36           5.25   
2      53482      8    10.0    321693          0  11050    36          13.65   
3      53483      8     7.0    231397          0  15350    36          14.20   
4      53484      7     7.0    171193          0   7800    36          10.00   
...      ...    ...     ...       ...        ...    ...   ...            ...   
20049  73529      0     8.0    601107          1  10000    12          18.85   
20050  73530      0    10.0    601102          1  10000    12          29.30   
20051  73531      0     4.0    601408          1  11000    12          24.75   
20052  73532      0     3.0    601904          1   8000    12          22.00   
20053  73533      2     1.0    601809          1   7000    12          14.95   

       issue_time  syndicated  installm

output_path :  test.dat.v10
process time :  2025-08-28 13:47:13
model_stacking_v10.1.ipynb
process_v10.ipynb
test.dat.v10
test.dat.v9
train.dat.v10
train.dat.v9


In [79]:
!wc -l $output_path

20055 test.dat.v10


In [80]:
print('done  time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))
print(df_concat.columns, '\n', df_concat.shape)
df_concat

done  time :  2025-08-28 13:47:14
Index(['id', 'title', 'career', 'zip_code', 'residence', 'loan', 'term',
       'interest_rate', 'issue_time', 'syndicated', 'installment',
       'record_time', 'history_time', 'total_accounts', 'balance_accounts',
       'balance_limit', 'balance', 'level', 'balance_account_avg',
       'loan_term_avg', 'balance_accounts_ratio', 'loan_log',
       'balance_account_avg_log', 'loan_term_avg_log',
       'balance_accounts_ratio_log', 'interest_rate_log', 'balance_log',
       'balance_limit_log', 'balance_accounts_log', 'level_hash', 'level_ord',
       'record_issue_diff', 'record_issue_days', 'record_history_diff',
       'record_history_days', 'issue_history_diff', 'issue_history_days',
       'tx_tmstp_max', 'tx_tmstp_min', 'tx_max_min_days', 'tx_count',
       'total_amount', '1_amount', '0_amount', 'tx_count_avg',
       'total_amount_avg', '1_amount_avg', '0_amount_avg', 'total_amount_avg2',
       '1_amount_avg2', '0_amount_avg2', 'tx_count_log'

,id,title,career,zip_code,residence,loan,term,interest_rate,issue_time,syndicated,installment,record_time,history_time,total_accounts,balance_accounts,balance_limit,balance,level,balance_account_avg,loan_term_avg,balance_accounts_ratio,loan_log,balance_account_avg_log,loan_term_avg_log,balance_accounts_ratio_log,interest_rate_log,balance_log,balance_limit_log,balance_accounts_log,level_hash,level_ord,record_issue_diff,record_issue_days,record_history_diff,record_history_days,issue_history_diff,issue_history_days,tx_tmstp_max,tx_tmstp_min,tx_max_min_days,tx_count,total_amount,1_amount,0_amount,tx_count_avg,total_amount_avg,1_amount_avg,0_amount_avg,total_amount_avg2,1_amount_avg2,0_amount_avg2,tx_count_log,total_amount_log,1_amount_log,0_amount_log,tx_count_avg_log,total_amount_avg_log,1_amount_avg_log,0_amount_avg_log,total_amount_avg2_log,1_amount_avg2_log,0_amount_avg2_log,tx_count_log_log
0,53480,10,5.0,512068,0,5950,36,14.10,1054600001,0,0,1054600080,795400642,17,9,10100.0,8233.00,B0,914.777778,165.277778,0.529412,8.691146,6.818681,5.107628,-0.635989,2.646175,9.015906,9.220291,2.197225,310,6.0,79,0.000914,259199438,0.000914,259199359,2999.992581,1.053302e+09,1.039046e+09,165.0,46.0,61815.52,11245.52,50570.00,0.278788,374.639515,68.154667,306.484848,1343.815652,244.467826,1099.347826,3.828641,11.031910,9.327725,10.831114,-1.277304,5.925964,4.221780,5.725168,7.203268,5.499084,7.002472,3.828641
1,53481,8,10.0,401852,2,15350,36,5.25,1172880047,0,0,1172882338,756003297,17,9,58600.0,23460.00,A0,2606.666667,426.388889,0.529412,9.638871,7.865828,6.055352,-0.635989,1.658228,10.063052,10.978490,2.197225,630,0.0,2291,0.026516,416879041,0.026516,416876750,4824.962384,1.170806e+09,1.158365e+09,144.0,56.0,38555.84,20161.94,18393.90,0.388889,267.748889,140.013472,127.735417,688.497143,360.034643,328.462500,4.025352,10.559863,9.911552,9.819774,-0.944462,5.590050,4.941739,4.849961,6.534511,5.886200,5.794423,4.025352
2,53482,8,10.0,321693,0,11050,36,13.65,1238631003,0,0,1277511093,658715225,17,9,15500.0,7853.00,B1,872.555556,306.944444,0.529412,9.310186,6.771426,5.726667,-0.635989,2.613740,8.968651,9.648595,2.197225,594,7.0,38880090,450.001042,618795868,450.001042,579915778,6711.988171,1.275350e+09,1.260490e+09,172.0,74.0,81589.80,44977.46,36612.34,0.430233,474.359302,261.496860,212.862442,1102.564865,607.803514,494.761351,4.304065,11.309460,10.713917,10.508141,-0.843429,6.161965,5.566422,5.360646,7.005394,6.409852,6.204076,4.304065
3,53483,8,7.0,231397,0,15350,36,14.20,1136161107,0,0,1198371199,774316843,17,9,10100.0,5815.00,B2,646.111111,426.388889,0.529412,9.638871,6.470971,6.055352,-0.635989,2.653242,8.668196,9.220291,2.197225,288,8.0,62210092,720.024213,424054356,720.024213,361844264,4188.012315,1.195776e+09,1.182038e+09,159.0,73.0,48205.32,24048.52,24156.80,0.459119,303.178113,151.248553,151.929560,660.346849,329.431781,330.915068,4.290459,10.783225,10.087829,10.092321,-0.778445,5.714320,5.018925,5.023417,6.492765,5.797369,5.801862,4.290459
4,53484,7,7.0,171193,0,7800,36,10.00,1214960354,0,1,1214957804,832207268,17,9,13100.0,8225.00,A3,913.888889,216.666667,0.529412,8.961879,6.817709,5.378360,-0.635989,2.302585,9.014934,9.480368,2.197225,17,3.0,-2550,-0.029514,382750536,-0.029514,382753086,4430.012569,1.214438e+09,1.199405e+09,174.0,91.0,78113.61,29797.86,48315.75,0.522989,448.928793,171.252069,277.676724,858.391319,327.449011,530.942308,4.510860,11.265920,10.302192,10.785513,-0.648196,6.106864,5.143137,5.626458,6.755060,5.791332,6.274653,4.510860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20049,73529,0,8.0,601107,1,10000,12,18.85,1130976000,0,0,1125964800,1018224000,6,3,3818.0,2224.69,A4,741.563333,833.333333,0.500000,9.210340,6.608761,6.725434,-0.693147,2.936513,7.707373,8.247482,1.098612,160,4.0,-5011200,-58.000000,1077408